In [1]:
code_dir = "../UXO_protected/+BTInvertPY"
import sys
sys.path.append(code_dir)

In [2]:
from BTSensor import SensorInfo, Model, preCalcLoopCorners, FModParam, forwardWithQ, sensorCoords2RxCoords, hprimary, formQmatrix
from BTInvert import getSettings, validate_std, signal2Noise, invert_single, invertML
import numpy as np
import os
import h5py
import matplotlib.pyplot as plt

In [3]:
data_directory = os.path.sep.join(["..", "data-blacktusk"])

In [4]:
# H5 functions
def proc_attr(inp):
    """HDF5 - Process attributes for the input group and return them in a dictionary."""
    dic = {}
    for att in inp.attrs.keys():
        if inp.attrs[att].dtype.char == 'S':
            dic[att] = [x.strip() for x in inp.attrs[att].tostring().decode('ascii').split(',')]
        else:
            dic[att] = inp.attrs[att][0] if isinstance(inp.attrs[att],np.ndarray) and inp.attrs[att].size==1 else inp.attrs[att]
    return dic
    pass

def proc_group(inp):
    """HDF5 - A recursive function for reading datasets and attributes into a dictionary."""
    dic = {}
    dic.update(proc_attr(inp))
    for key in inp.keys():
        if isinstance(inp[key], h5py.Group):
            dic.update({key:proc_group(inp[key])})
        else:
            dic[key] = inp[key][()]
        pass
    return dic

In [5]:
# Load the data file and read all data into a dictionary
# dfile = os.path.join('.','matlab','d190723_BlossomPointPOS_inv.h5')
dfile = os.path.join(data_directory, 'Testplot5F.h5')
f = h5py.File(dfile, 'r')
dic = proc_group(f)
f.close()
 
icell = '1'
iinv = '1' # it is common for the 1st inversion result to be 1-obj, 2nd: 2-obj and so on

OSError: Unable to open file (unable to open file: name = '../data-blacktusk/Testplot5F.h5', errno = 2, error message = 'No such file or directory', flags = 0, o_flags = 0)

In [ ]:
dic.keys()

In [ ]:
# Generate the sensorinfo structure
sensor_name = dic['SensorName'][0]
sensor_config = dic['SensorConfig'][0]
afile = os.path.join('..',"+BTInvertPY",'config','sensor_definitions','{}___{}.yaml'.format(sensor_name,sensor_config))
sensorinfo = SensorInfo.fromYAML(afile)[0]

print(sensor_name)

XYZ Data.  This is not used in the script but you will want to use it for any target picking you do.
The same channels are in the cell_data below.

In [ ]:
xyz_dict = dic["XYZ"]
xyz_ch = xyz_dict['Info']['ChannelNames']
xyz_datai = xyz_dict['Info']['Data']['ChannelIndex'].flatten().astype(int)-1 # These are indices to the decay data
xyz_yawi = int(xyz_dict['Info']['Yaw']['ChannelIndex'])-1 # index to the yaw channel
xyz_mni = int(xyz_dict["Info"]["MeasNum"]["ChannelIndex"])-1

times = dic['SensorTimes'].flatten()

All the required channels in uxolab are registered in dic['XYZ']['Info'] similar to above.
Channel names can change from dataset to dataset so this registration is a reliable way to find a particular channel.


In [ ]:
xyz_data = dic['XYZ']['Data']

In [ ]:
len(xyz_ch)

In [ ]:
xyz_data[xyz_mni, :]

In [ ]:
"""
CELL Data
"""
cell_target = dic['CELL'][icell]['Target'].flatten()
cell_data = dic['CELL'][icell]['Data']
cell_std = dic['CELL'][icell]['CellStd']
cell_ch = dic['CELLInfo']['ChannelNames']




In [ ]:
""" explanation of the line below
UXOLab datasets can store 1 or more inversion results, when there is two or more results
the field is accessed with the additional key iinv, when only one result is present the key is not needed.
"""
inv_res = dic['CELL'][icell]['InvResults'][iinv] if '1' in dic['CELL'][icell]['InvResults'].keys() else dic['CELL'][icell]['InvResults']

inv_times = inv_res['Invert']['Times'].flatten()
cell_height = inv_res['Invert']['Height']
cell_final_mod = inv_res['ModelFinal']
cell_min_s2n = inv_res['Invert']['min_S2N']
inv_ch = inv_res['Invert']['Channels']


btinvert_settings = inv_res['Invert']['BTInvert']
# these lines are formating the datatypes, so that the settings dict is as expected by the inversion API
btinvert_settings = {k:(v[0] if isinstance(v,list) and isinstance(v[0],str) else v) for (k,v) in btinvert_settings.items()}
btinvert_settings = {k:(v[0] if isinstance(v,np.ndarray) and v.size == 1 else v) for (k,v) in btinvert_settings.items()}
btinvert_settings = {k:(v.flatten().tolist() if isinstance(v,np.ndarray) else v) for (k,v) in btinvert_settings.items()}
btinvert_settings = {k:(None if v!=v else v) for (k,v) in btinvert_settings.items()}

# select only those data that were inverted. Data is limited in a cell using a polygon mask in uxolab.
mask_ind = inv_res['MaskIndices'].astype(int).flatten() - 1 # these are 1-based indices from matlab
cell_data = cell_data[:,mask_ind].T

In [ ]:
"""
Find the indices of various required channels.
"""
timesi = np.isin(times,inv_times)
datai = np.isin(cell_ch,inv_ch).nonzero()[0]
mni = int(dic['CELLInfo']['MeasNum']['ChannelIndex'])-1
pitchi = int(dic['CELLInfo']['Pitch']['ChannelIndex'])-1
rolli = int(dic['CELLInfo']['Roll']['ChannelIndex'])-1
yawi = int(dic['CELLInfo']['Yaw']['ChannelIndex'])-1
easti = int(dic['CELLInfo']['Easting']['ChannelIndex'])-1
northi = int(dic['CELLInfo']['Northing']['ChannelIndex'])-1
elevi = int(dic['CELLInfo']['Elevation']['ChannelIndex'])-1

# I adjust the data coordinates origin to the cell target.  Also the median elevation is subtracted and then the sensor height is added to form a z-channel
pos = cell_data[:, [easti,northi,elevi]] - np.append(cell_target,np.median(cell_data[:,elevi])-cell_height)
pitch = np.deg2rad(cell_data[:, pitchi])
roll = np.deg2rad(cell_data[:, rolli])
yaw = np.deg2rad(cell_data[:, yawi])
mn = cell_data[:, mni].astype(int)-1
V = cell_data[:,datai]

# This function is for checking the supplied data errors estimate and formating it
stdv = validate_std(cell_std[timesi,:].T,mn,V.shape[1])

# Get default inversion settings and update them with those used in UXOLab
settings = getSettings(sensorinfo.name)
settings.update(btinvert_settings)
settings.update({'min_s2n':cell_min_s2n})

# Compute S2N and combine percent error with data errors.
s2n_vec,s2n_mask = signal2Noise(sensorinfo.name,inv_times,stdv,V,settings)
d_f = 1./(stdv + abs(V)*settings["percent_error"])
d_f *= s2n_mask

In [ ]:
mni

In [ ]:
# Call the inversion API
result = invertML(  data = V,
                    mnum = mn,
                    pos = pos,
                    pitch = pitch,
                    roll = roll,
                    yaw = yaw,
                    times = inv_times,
                    s2n_vec = s2n_vec,
                    s2n_mask = s2n_mask,
                    stdv = stdv,
                    d_f = d_f,
                    sensorinfo = sensorinfo,
                    settings = settings
)

In [ ]:
# Take the UXOLab inversion result from the H5 file and create a Model object for easy printing
aux = np.reshape(cell_final_mod[6:],(3,int((cell_final_mod.size-6)/3))).T
aux_xyz = cell_final_mod[0:3].flatten() - np.append(cell_target,0)
uxolab_final_model = Model(xyz = aux_xyz, gba = cell_final_mod[3:6].flatten(),l1 = aux[:,2],l2 = aux[:,1],l3 = aux[:,0],times = inv_times)

# This is our final result from the inversion API above
btinvert_final_model = result['model'][0][0]

In [ ]:
# Print out the inversion results.  They should be close to identical.
print('UXOLAB RESULT\n{}\n'.format(uxolab_final_model))
print('BTINVERT RESULT\n{}\n'.format(btinvert_final_model))

In [ ]:
xyz_data.shape

In [ ]:
print([xyz_ch[i] for i in xyz_datai])

In [ ]:
xyz_data[np.where(xyz_ch=="tearly")[0], 0:30]

In [ ]:
xyz_data

In [ ]:
xyz_ch